In [1]:
from OPTIMIZED_whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time

In [2]:
from whole_neuron_classifier_datajoint_adapted import write_Whole_Neuron_Off_file

In [3]:
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config["display.limit"] = 20
    
#schema = dj.schema('microns_ta3p100')
#ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")


Connecting celiib@10.28.0.34:3306


In [4]:
key = dict(segmentation=3,segment_id=648518346349475536)

In [5]:
start_time = time.time()

print("\n\n*****Starting Auto Labeling for " + str(key["segment_id"]) + "******")

segment_id = key["segment_id"]

#get the vertices and faces from datajoint
# get the newly stitched mesh
# get the original mesh


verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & key).fetch1("vertices","triangles")

cgal_location = "/notebooks/Pass_2_Excitatory_4_Auto_Classifier_Whole_Neuron_Run_2/automatic_classifier_revised_efficiency/temp/"
cgal_file_sdf = "648518346349475536_fixed-cgal_3_0.20_sdf.csv"
cgal_file_seg = "648518346349475536_fixed-cgal_3_0.20.csv"

#run the whole algorithm on the neuron to test
verts_labels, faces_labels = extract_branches_whole_neuron(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                     triangles=faces,pymeshfix_Flag=False,
                     import_CGAL_Flag=True,
                     import_CGAL_paths = [cgal_location + cgal_file_seg,
                                         cgal_location + cgal_file_sdf],
                     return_Only_Labels=True,
                     clusters=3,
                     smoothness=0.2)

print(f"Finished Auto Labeling: {time.time() - start_time}")




*****Starting Auto Labeling for 648518346349475536******
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.18535876274108887
2) Staring: Generating CGAL segmentation for neuron
2) Finished: Generating CGAL segmentation for neuron: 40.93231272697449
3) Staring: Generating Graph Structure and Identifying Soma
soma_index = 0
3) Finished: Generating Graph Structure and Identifying Soma: 1.072035789489746
5) Staring: Finding Apical Index
Soma Index = 0
Soma Connections = [1, 4, 16, 12, 27]
soma_80_percent = 249892.48125
possible_Axons_filter_1 = [1]
size_threshold_filter = [1]
height_threshold_filter = [1]
sdf_threshold_filter = [1]
possible_Axons_filter_2 = [1]
possible_Apical = 1
5) Finished: Finding Apical Index: 2.0870752334594727
6) Staring: Classifying Entire Neuron
27 = stub soma
13 = stub apical
18 = stub apical
20 = stub apical
21 = stub apical
22 = stub apical
23 = stub apical
24 = stub apical


In [12]:
#export the labels
labels = faces_labels.astype("int")
np.savetxt("temp/" + str(key["segment_id"]) + '_output_labels.csv', labels, fmt='%d', delimiter='\n')

In [10]:
labels

array([5, 2, 2, ..., 3, 3, 3])